IBMQ (IBM Quantum Provider) is deprecated.

In [1]:
from qiskit import IBMQ
from qiskit.providers.ibmq import least_busy
from qiskit import QuantumCircuit, transpile

qc = QuantumCircuit(3, 3)
qc.h([0,1,2])

qc.measure([0,1,2], [0,1,2])
qc.draw()

┌───┐┌─┐      
q_0: ┤ H ├┤M├──────
     ├───┤└╥┘┌─┐   
q_1: ┤ H ├─╫─┤M├───
     ├───┤ ║ └╥┘┌─┐
q_2: ┤ H ├─╫──╫─┤M├
     └───┘ ║  ║ └╥┘
c: 3/══════╩══╩══╩═
           0  1  2

In [12]:
# Get the API token in https://quantum-computing.ibm.com/
#IBMQ.save_account("token",overwrite=True)

if not IBMQ.active_account():
    provider = IBMQ.load_account()

save_account stores the API token to disk, hence it must be done only once after rotating the token. See https://github.com/Qiskit/qiskit-ibmq-provider for more details.

load_account must be done once at the beginning of a new session. Otherwise we'll get a warning that the existing account in the session will be replaced. We combine the call with a check if the account is already active in the session.

Get all available backends for this session:

In [13]:
provider.backends()

[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_lima') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_belem') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_quito') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQSimulator('simulator_statevector') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQSimulator('simulator_mps') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQSimulator('simulator_extended_stabilizer') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQSimulator('simulator_stabilizer') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_manila') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibm_nairobi') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibm_oslo') from IBMQ(hub='ibm-q', group='open', project='main')>]

In [18]:
# Select a backend (real QC hardware)

#backend = provider.get_backend('ibmq_quito')

backend = least_busy(provider.backends(simulator=False))
print("Least busy backend:", backend.name())

Least busy backend: ibmq_belem


In [19]:
# Prepare the circuit to run

optimized_circuit = transpile(qc, backend)
optimized_circuit.draw()

global phase: 3π/4
               ┌─────────┐┌────┐┌─────────┐┌─┐      
      q_0 -> 0 ┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├┤M├──────
               ├─────────┤├────┤├─────────┤└╥┘┌─┐   
      q_1 -> 1 ┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├─╫─┤M├───
               ├─────────┤├────┤├─────────┤ ║ └╥┘┌─┐
      q_2 -> 2 ┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├─╫──╫─┤M├
               └─────────┘└────┘└─────────┘ ║  ║ └╥┘
ancilla_0 -> 3 ─────────────────────────────╫──╫──╫─
                                            ║  ║  ║ 
ancilla_1 -> 4 ─────────────────────────────╫──╫──╫─
                                            ║  ║  ║ 
          c: 3/═════════════════════════════╩══╩══╩═
                                            0  1  2

In [20]:
# Run the job on the selected backend

job = backend.run(optimized_circuit,shots=1000)
retrieved_job = backend.retrieve_job(job.job_id())
result = retrieved_job.result()
print(result.get_counts())

{'000': 138, '001': 144, '010': 126, '011': 127, '100': 121, '101': 123, '110': 116, '111': 105}
